In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import networkx as nx
import json

In [2]:
cyto.load_extra_layouts()

# Muhaddithat App

This notebook contains all application code for the Muhaddithat dashboard with documentation in markdown. We also have the same code without Jupyter formatting in the file `app.py`, which can be used for deployment on a server.

To start the app, click 'run all cells' and visit http://127.0.0.1:8050/ in your web browser.
Alternatively, you can open a terminal window, navigate to the project directory, and run `python app.py`. If you use the latter method, make sure you are within a Python environment with all requisite packages installed.

In [3]:
app = JupyterDash(__name__)

## Data

In [4]:
#filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-3_SEEDS-53-54-56-59-69-70-71-84-244-2802-10526-10737-11039-11457.cyjs'
filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-1_SEEDS-11457.cyjs'

with open(filename, 'r') as f:
    data = json.load(f)

In [5]:
filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-1_SEEDS-53.cyjs'

with open(filename, 'r') as f:
    data2 = json.load(f)

## Styling

In [6]:
default_stylesheet = [
    {
        'selector': '[gender="m"]',
        'style': {
            'background-color': '#7b7b7b',
            'background-opacity': '0.5',
            'width':'0.01px',
            'height':'0.01px',
            'padding':'3px'
        }
    },
    {
        'selector':'[gender="f"]',
        'style':{
            'background-color':'#ff0000',
            'width':'0.2px',
            'height':'0.2px',
            'padding':'3px'
        }        
    },
    {
        'selector':'[id="1"]',
        'style':{
            'background-color':'#8fce00',
            'width':'5px',
            'height':'5px',
            'background-opacity':'1'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    }    
]

## Layout

In [7]:
app.layout = html.Div([
    html.Div([
    dcc.Dropdown(
        id='dropdown1',
        options=[{'label': 'Aishah bint Abi Bakr', 'value': '53'} ,
                 {'label': 'Umm al-Darda as-Sughra','value': '11457'}],
        value='53',    
        ),],
        style={'width': '20%', 'display': 'inline-block'}
    ),
    
    # section for displaying the graph 
    cyto.Cytoscape(
        id='cytoscape',
        autoungrabify=True, # Prevent users from moving nodes around.
        elements= data['elements'], #edges + nodes,
        layout={
            'name': 'breadthfirst',
            'directed': 'true',
            'maximal': 'true',
            'roots': '1', 
            'circle':'true'
        },
        style={'width': '100%', 'height': '500px'},
        stylesheet=default_stylesheet
    ),
    # section for node data output
    html.P("Narrator Info:"),
     html.P(id='cytoscape-tapNodeData-output')
#    dcc.Markdown(id='cytoscape-selectedNodeData-markdown')
])

## Callbacks

https://dash.plotly.com/cytoscape/events


https://dash.plotly.com/cytoscape/callbacks


https://github.com/27shraddhaS/dropdown-graphs-dash/blob/main/dash-dropdown-graph.ipynb



@app.callback(Output('cytoscape-selectedNodeData-markdown', 'children'),
              Input('cytoscape-event-callbacks-3', 'selectedNodeData'))

def displaySelectedNodeData(data_list):
    if data_list is None:
        return "No narrator selected."

    narrators_list = [data['label'] for data in data_list]
    return "You selected the following narrators: " + "\n* ".join(narrators_list)

In [8]:
@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently clicked/tapped the narrator: " + data['name']
  
    
# change the graph to view the narrator selected in the dropdown menu

@app.callback(Output('cytoscape', 'elements'),
              Input('dropdown1', 'value'))

def update_elements(value): 
    if value == '11457':
        return data['elements']
    if value == '53':
        return data2['elements']
    
# Update the stylesheet so that the edges of the selected narrator are colored 

@app.callback(Output('cytoscape', 'stylesheet'),
              Input('dropdown1', 'value'))

def update_stylesheet(value):
    new_styles = []
    if value == '11457':
        new_styles = [
             {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    },  
            {
          'selector':  '[source="11457"]',
          'style':{
          'line-color':'#ff0000',
          'width':'0.1px',
          'opacity':'1',
          'target-arrow-color':'#ff0000'} 
        }]
    if value == '53':
        new_styles = [
             {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    },  {
          'selector':  '[source="53"]',
          'style':{
          'line-color':'#ff0000',
          'width':'0.1px',
          'opacity':'1',
          'target-arrow-color':'#ff0000'} 
        }]
    return default_stylesheet + new_styles

## Run App

In [9]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
